In [ ]:
from PIL import Image
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from dask.distributed import Client, progress
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

In [ ]:
def sparseH(L, J, h, periodic):
    
    """
    generates the sparse Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (csr_matrix): sparse matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    # initialize 
    H_data = []
    H_rows = []
    H_cols = []
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1, L + 1): # iterate over all sites
            
            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            "Keep track of the indices with non-zero matrix elements"
            
            H_data.append(-h)
            H_rows.append(alpha)
            H_cols.append(beta)
    
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim):  # iterate over all states
        
        A = 0
        
        for j in range(1, L): # iterate over all sites
            
            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
                
        "Handling periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
        
        if A != 0: # Check if the resulting matrix element is non-zero, if so, keep track of it
        
            H_data.append(A)
            H_rows.append(alpha)
            H_cols.append(alpha)

    H_data = np.array(H_data, dtype=float) # convert the list into a np array
    
    H = scipy.sparse.csr_matrix((H_data, (H_rows, H_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix
    
    return H

In [ ]:
def H_psi(psi, L, J, h, periodic):
    
    """
    Generates Hpsi given psi
    """
    dim = 2**L
    Hpsi = np.zeros(dim, dtype=np.float64)


    "Calculation of off-diagonal elements due to the magnetic field"

    for beta in range(dim): # iterate over all states

        for j in range(1, L + 1): # iterate over all sites

            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip

            Hpsi[alpha] -= h * psi[beta] # change the coefficient of the alpha state


    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim): # iterate over all states

        A = 0

        for j in range(1, L): # iterate over all sites

            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j):  # check if site j and j+1 have the same spin

                A -= J # if they do, decrease the energy by the ising interaction term

            else:
                A += J # if not, increase the energy by the ising interaction term

        if periodic and L > 1:  # L > 1 needed for periodicity to mean anything

            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin

                A -= J # if they do, decrease the energy by the ising interaction term

            else:
                A += J # if not, increase the energy by the ising interaction term

        Hpsi[alpha] += A * psi[alpha] # change the coefficient of the alpha state
        
    return Hpsi


In [ ]:
def psi_gs(L, J, h, periodic):
    H = sparseH(L, J, h, periodic)
    return scipy.sparse.linalg.eigsh(H, k=1, which='SA')

In [ ]:
def schmidt_psi(k, L, J, h, periodic):
    
    l = L//2
    eig, vec = psi_gs(L, J, h, periodic)
    vec = vec.flatten()
    
    psi_matrix = vec.reshape((2**l, 2**(L-l)))
    U, S, Vh = np.linalg.svd(psi_matrix, compute_uv=True)
    
    S_truncated = S[:k]
    U_truncated = U[:, :k]
    Vh_truncated = Vh[:k, :]

    psi_approx = np.dot(U_truncated, np.dot(np.diag(S_truncated), Vh_truncated))
    psi_approx_flat = psi_approx.flatten()

    Hpsi_approx = H_psi(psi_approx_flat, L, J, h, periodic)
    energy_approx = np.dot(psi_approx_flat.conj().T, Hpsi_approx)
    norm_psi_approx = np.linalg.norm(psi_approx_flat)**2
    normalized_energy_approx = energy_approx / norm_psi_approx
    d_k = np.sqrt(np.sum(S[k:]**2))
    
    return d_k, normalized_energy_approx-eig[0] 

In [ ]:
schmidt_psi(2, 10, 1, 1, True)

In [ ]:
# Parameters
L = 10
J = 1
h = 1
periodic = False

# Gather the data
d_values = []
delta_E_values = []

k_values = range(1, 2 ** (L // 2) + 1)

for k in k_values:
    d_k, delta_E = schmidt_psi(k, L, J, h, periodic)
    d_values.append(d_k)
    delta_E_values.append(delta_E)

# Fit a quadratic curve (degree 2)
coeffs = np.polyfit(d_values, delta_E_values, 2)
quadratic_fit = np.poly1d(coeffs)

d_values_cont=np.linspace(d_values[0], d_values[-1], 1000)
# Plot data points and the fitted curve
plt.figure(figsize=(10, 6))
plt.plot(d_values, delta_E_values, marker='o', linestyle='-', color='blue', label='Data')
plt.plot(d_values_cont, quadratic_fit(d_values_cont), color='red', linestyle='--', label=f'Fit: {coeffs[0]:.2f} d^2 + {coeffs[1]:.2f} d')
plt.title('Truncation Error vs Discarded Norm with Quadratic Fit')
plt.xlabel('Discarded Norm d(k)')
plt.ylabel('Energy Error $\\Delta E(k)$')
plt.legend()
plt.savefig(os.path.join(directory, 'quadraticfit.png'), dpi = 400)
plt.grid(True)
plt.show()